In [1]:
import numpy as np
import numba as nb
import numba.cuda as cuda
from numba import guvectorize, vectorize
from numba.cuda.random import xoroshiro128p_normal_float32, create_xoroshiro128p_states

# from user import BLOCKS, DIM, THREADS
from config import D, M

from user import a, b

In [2]:
from core import multiply_matrix_by_scalar, add_inplace, multiply_matrix,\
    gen_normal_float32
from config import D, M

In [46]:
@cuda.jit(device=True)
def euler_step(point, time, dt, a_func, b_func, state):
    temp_dt = cuda.local.array(shape=(D, 1), dtype=point.dtype)
    temp_dw = cuda.local.array(shape=(D, M), dtype=point.dtype)
    dw = cuda.local.array(shape=(M, 1), dtype=point.dtype)
    a_func(time, point, temp_dt)  # save result in temp
    multiply_matrix_by_scalar(temp_dt, dt)
    b_func(time, point, temp_dw)
    add_inplace(point, temp_dt)
    gen_normal_float32(dw, state)
    multiply_matrix_by_scalar(dw, dt)
    multiply_matrix(temp_dw, dw, temp_dt)
    add_inplace(point, temp_dt)

In [47]:
@cuda.jit(device=True)
def b(t, x, result):
    for i in range(D):
        for j in range(M):
            result[i, j] = 5

In [48]:
@cuda.jit
def kernel(vector, state):
    euler_step(vector, 3, 1.e-4, a, b, state)

In [49]:
THREADS, BLOCKS = 1, 1

In [50]:
point = 2 * np.ones(shape=(D, 1)).astype('float32')

d_point = cuda.to_device(point)
rng_state = create_xoroshiro128p_states(THREADS * BLOCKS, seed=2)

kernel[THREADS, BLOCKS](d_point, rng_state)

d_point.copy_to_host(point)

C:\Users\marcin\PycharmProjects\cuSDE\venv\lib\site-packages\numba\cuda\compiler.py:726: NumbaPerformanceWarning: Grid size (1) < 2 * SM count (8) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


array([[1.9992927],
       [1.9992927],
       [1.9992927]], dtype=float32)

In [7]:
point

array([[1.],
       [1.],
       [1.]], dtype=float32)

In [ ]:
d_start = np.ones(shape=(2, 4)).astype('float32')
result = np.zeros_like(d_start)
d_result = cuda.to_device(result)
d_ary = cuda.to_device(d_start)
rng_states = create_xoroshiro128p_states(THREADS * BLOCKS, seed=2)
caller[1, 1](d_ary, d_result)
d_result.copy_to_host(result)